In [53]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input,Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

## Engineering Features

In [2]:
rootFolder = "bird_data"
wavFolder = os.path.join("bird_data","wav_files")
wavFiles = os.listdir(wavFolder)
bird_calls_df = pd.read_csv(os.path.join(rootFolder, "bird_calls.csv"))
bird_calls_df.head()

,fname,label
0,American_Goldfinch_XC114342.wav,American_Goldfinch
1,American_Goldfinch_XC124312.wav,American_Goldfinch
2,American_Goldfinch_XC133564.wav,American_Goldfinch
3,American_Goldfinch_XC133565.wav,American_Goldfinch
4,American_Goldfinch_XC141469.wav,American_Goldfinch


In [3]:
mel_features_df = pd.read_csv(os.path.join(rootFolder, "mel_features.csv"))
mfcc_features_df = pd.read_csv(os.path.join(rootFolder, "mfcc_features.csv"))

In [4]:
mel_features_df.head()

,label,feature
0,American_Goldfinch,[[-80. -80. -80. ... -80. -80. -80.]\n [-80. -...
1,American_Goldfinch,[[-80. -80. -80. ... -80. -80. -80.]\n [-80. -...
2,American_Goldfinch,[[-80. -80. -80. ... -...
3,American_Goldfinch,[[-80. -80. -80. ... -...
4,American_Goldfinch,[[-80. -80. -80. ... -80. -80. -80.]\n [-80. -...


In [5]:
mfcc_features_df.head()

,label,feature
0,American_Goldfinch,[[ -2.48275794 -3.56181979 -2.70022824 ... ...
1,American_Goldfinch,[[ 0. -1.13543094 -0.34263572 ... ...
2,American_Goldfinch,[[ -1.06088357 -5.78667663 -4.09501159 ... ...
3,American_Goldfinch,[[ -0.7841282 -8.00023103 -6.01809665 ... ...
4,American_Goldfinch,[[ -1.33764821 -1.20825628 -0.57548347 ... ...


### Train and Test datasets

In [191]:
# Function to create train and test dataframe from shuffled features
def test_train(df, train_size=80):
    classes = list(np.unique(df.label))
    train = []
    test = []
    for c in classes:
        temp = df[df['label']==c].sample(frac=1)
        train.append(temp.head(train_size))
        test.append(temp.tail(100-train_size))
    train_df=pd.concat(train[:10]).sample(frac=1).reset_index(drop=True)
    test_df = pd.concat(test[:10]).sample(frac=1).reset_index(drop=True)
    return train_df, test_df
        

In [192]:
mfcc_train, mfcc_test = test_train(mfcc_features_df)

In [193]:
mfcc_train.head()

,label,feature
0,American_Robin,[[ -4.08599101 -3.3323396 -1.91455583 ... ...
1,Blue-grey_Gnatcatcher,[[ 0. -6.58899379 -5.77285376 ... ...
2,Carolina_Chickadee,[[ 0.00000000e+00 -8.41467641e-02 -5.76858570e...
3,Blue-grey_Gnatcatcher,[[ -9.07276724 -7.36076227 -5.05184096 ... ...
4,Cedar_Waxwing,[[ 0. 0. 0. ... 0. 0. 0.]\n [-80. -...


In [194]:
mfcc_test.head()

,label,feature
0,Blue-grey_Gnatcatcher,[[ 0.00000000e+00 -1.09809000e+00 -2.28547934e...
1,Barn_Swallow,[[ -1.999076 -1.41482581 -1.02702158 ... ...
2,Carolina_Chickadee,[[ 0. 0. 0. ... 0. 0. 0.]\n [-80. -...
3,Barn_Swallow,[[ 0. -6.41529659 -4.65937475 ... ...
4,Ruby-crowned_Kinglet,[[ 0. -5.9585571 -3.52180889 ... ...


# Convolutional Neural Network

In [10]:
# Batch and target size
size_batch = 32
size_target = (288,288)


In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(288, 288, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 286, 286, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 284, 284, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 142, 142, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 142, 142, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1290496)           0         
_________________________________________________________________
dense (Dense)                (None, 64)                82591808  
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
callbacks = [early_stopping]